In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageFilter
import pytesseract
from scipy.ndimage import *
import scipy.signal as scisignal
import skimage.exposure as expo
import skimage.morphology as morpha
from skimage.measure import *
import scipy.ndimage as ndimg
import supported_functions as sf
import create_templates as ct

In [12]:
aspect_ratio = lambda image: image.size[0] / image.size[1]

In [13]:
# Import image
image = Image.open("data/71tfK2KxQ-L._AC_SX466_.jpg")

# Resize the image while maintaining aspect ratio
new_size_of_image = (400,round(400/aspect_ratio(image)))
new_image = image.resize(new_size_of_image)

# Convert Image to Grayscale (Colour Image Processing)
image_grayscale = new_image.convert("L")

# Perform median filtering on the image
median_filtered_image = image_grayscale.filter(ImageFilter.MedianFilter(3))

# Structural element for image dilation and erosion
structural_element = np.array([[0,1,0],
                      [1,1,1],
                      [0,1,0]])

# Converting median filtered image to its binary form for image dilation and erosion
binary_image = median_filtered_image.convert("1")

# Performing Image Dilation using the structural element
dilated_image = np.int64(ndimg.binary_dilation(input=binary_image,structure=structural_element))

# Performing Image Erosion using the structural element
eroded_image = np.int64(ndimg.binary_erosion(input=binary_image,structure=structural_element))

# Morphological Element for enhancing edges of the image
gradient_image = dilated_image - eroded_image

# Scaling the image to [0,1] to convert the image to double precision
gradient_image_scaled = gradient_image / gradient_image.max()

# Convolution of double precision image for beightening the edges
kernel = [[1,1],
          [1,1]]
convolved_image = scisignal.convolve2d(in1=gradient_image_scaled,in2=kernel,mode="valid")

# Intensity scaling between the range 0 to 1
low_in = 0.5
high_in = 0.7
low_out = 0
high_out = 1
gamma = 0.1
gain = (high_out - low_out) / (high_in - low_in)
intensity_scaled_image = expo.adjust_gamma(image=convolved_image,gamma=gamma,gain=gain)

# Conversion of double image back to binary image
back_binary_image = np.round(intensity_scaled_image / intensity_scaled_image.max())

In [14]:
back_binary_image

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [15]:
# Elimination of edges of the licence plate

eroded_back_binary_image = np.float64(ndimg.binary_erosion(input=back_binary_image,structure=np.ones(shape=(1,50))))
eliminated_edges_image = back_binary_image - eroded_back_binary_image

# Filling all the regions of the image with holes
eliminated_edges_image_inverted = 1 - eliminated_edges_image
filled_image_with_holes_before_invert = ndimg.binary_fill_holes(eliminated_edges_image_inverted)
filled_image_with_holes = 1 - filled_image_with_holes_before_invert

# Thinning the image filled with holes to make sure that characters do not match with each other (THIS STEP IS THE ACTUAL STEP FOR MORPHOLOGICAL PROCESSING
thinned_image = np.float64(morpha.thin(filled_image_with_holes))
eroded_thinned_image = np.float64(ndimg.binary_erosion(input=thinned_image,structure=np.ones(shape=(3,1))))

In [16]:
# Selecting all the pixels of area more than 100 (Image masking)
image_with_labels, number_of_labels = ndimg.label(eroded_thinned_image)
area_of_each_component = np.bincount(image_with_labels.ravel())
mask_with_pixel_areas_greater_than_100 = area_of_each_component >= 100
image_with_pixel_areas_greater_than_100 = np.float64(mask_with_pixel_areas_greater_than_100[image_with_labels])
final_image_for_vehicle_number_detection = np.int64(image_with_pixel_areas_greater_than_100)

In [17]:
# Considering two properties of image regions: 1) "Bounding Boxes" and 2) "Binary Images" corresponding to Bounding Images
image_region_properties = regionprops(final_image_for_vehicle_number_detection)

# Considering the bounding boxes in a matrix of order <Number of Bounding Boxes> X 4;
bounding_boxes = np.vstack([i.bbox for i in image_region_properties])

In [18]:
cc = ct.create_templates()

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 33 and the array at index 1 has size 17

In [9]:
# Retrieving an array comprising the indices for bounding boxes needed for character extraction
character_extraction_bounding_box_indices = sf.controlling(bounding_boxes)

IndexError: index 0 is out of bounds for axis 0 with size 0